<a href="https://colab.research.google.com/github/MaricelaMH/SIMULACION-II/blob/main/Frecuentistas_vs_Bayesianos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # F R E C U E N T I S T A S   _  V S  _  B A Y E S I A N O S

 Principalmente el desacuerdo de los frecuentistas con los bayesianos es la definición de probabilidad.

1. **Definición frecuentista de la probabilidad:** Sostiene que la
probabilidad de un evento es el límite al que tiende la frecuencia relativa de ocurrencia de dicho evento cuando el número de ensayos se incrementa indefinidamente. Es decir, según los frecuentistas, la probabilidad se basa en la observación empírica de eventos repetidos bajo condiciones similares.

* Si realizamos un experimento muchas veces, la probabilidad de un evento $A$ se define como el valor al que tiende la proporción de veces que ocurre $A$ en relación al número total de ensayos.

* Matemáticamente, se expresa como:

\\
 $$ P(A) = \lim_{n \to ∞ } ( \frac{\text{Número de veces que ocurre el evento $A$}}{\text{Número total de repeticiones del experimento}} )$$

\\
2. **Definición bayesiana de la probabilidad:** La probabilidad en el enfoque bayesiano se basa en el teorema de Bayes, que establece cómo actualizar las creencias o probabilidades iniciales (llamadas probabilidades a priori) cuando se adquieren nuevos datos o evidencia, resultando en una nueva probabilidad a posteriori.

* El teorema de Bayes se expresa matemáticamente como:

$$P(A|B)=\frac{P(B|A)* P(A)}{P(B)}$$

Donde:

* $P(A∣B)$ es la probabilidad de $A$ dado que se ha observado $B$ (la probabilidad a posteriori).

* $P(B∣A)$ es la probabilidad de observar $B$ si $A$ es verdadero.

* $P(A)$ es la probabilidad de $A$ antes de observar $B$ (la probabilidad a priori)

* $P(B)$ es la probabilidad total de observar $B$.

Esta diferencia que podría decirse que es sutil, puede llevar, en la práctica, a enfoques muy diferentes para el análisis estadístico de datos. A continuación, exploraremos algunos ejemplos elegidos para ilustrar las diferencias de enfoque, junto con el código Python asociado para demostrar los aspectos prácticos de los enfoques frecuentista y bayesiano.



# Ejemplo 1: Mediciones del flujo de fotones
Apuntamos un telescopio al cielo y observamos la luz que proviene de una sola estrella. Para simplificar, supondremos que el flujo fotónico real de la estrella es constante con el tiempo, es decir ,que tiene un valor fijo $F$, también ignoraremos efectos como los errores sistemáticos del fondo del cielo. Supondremos que se realizan una serie de $N$ mediciones, donde la i-ésima medición informa el flujo observado $F_i$ y el error $e_i$. La pregunta es dado este conjunto de mediciones $D = \{F_i, e_i\}$, ¿cuál es nuestra mejor estimación del flujo verdadero $F$?